In [4]:
#Optimal battery sizing
#Om Vinayaka

import Pkg; Pkg.add("CSV")
using JuMP
using LinearAlgebra
using GLPK
using Pkg
using CSV
using DataFrames; 

#Loading wind power,solar power and load data
dt=CSV.read("V:\\PHD\\Stochastic_Process\\Project_details\\Literature\\Battery_sizing_data.csv", DataFrame)

W_P=(dt[!,1]*1.0);S_P=(dt[!,2]*1.0);L_P=(dt[!,3]*0.1);

len=size(W_P,1)
D_P=zeros(len)

#Excess avalibaility or deficiency of renewable energy
for i in 1:len
   D_P[i]=round(W_P[i].+S_P[i].-L_P[i])
end


pi_z=zeros(w)
pi_m=zeros(w)
B_C=1


#Range of batteries
w=100;
for C in 1:w
function solve_opt(B_C)
 opt = Model(GLPK.Optimizer) 
 @variable(opt,0<=pi_s[i=1:1,j=1:(C.+1)])
 @variable(opt,0<=pi_z[i=1:1,j=1:(C)])   
 #@objective(opt, Min, (C*B_C))
   
  T_P=zeros(C.+1,C.+1)
  X=zeros(len)

#DTMC modeling and transition probability matrix        
   for i in 1:len 
    if i==1       
     X[i]=min(C,max(0,(C).+D_P[i]))
     a=convert(Int64,X[i])
     b=convert(Int64,(C))
     T_P[b.+1,a.+1]+=1
    else
     X[i]=min(C,max(0,X[i-1].+D_P[i]))
     a=convert(Int64,X[i])
     b=convert(Int64,X[i-1])
     T_P[b.+1,a.+1]+=1    
    end      
   end


#Verification for tranisition probability matrix
   sum=0;
   for i=1:(C+1)
    for j=1:(C+1)
      sum=sum.+T_P[i,j]
    end
   end

 T_PM=zeros(C+1,C+1)

   for i=1:(C+1)
    for j=1:(C+1)
     d=0
     for k=1:(C+1)
      d=d.+T_P[i,k]        
     end
    T_PM[i,j]=(T_P[i,j]./d)     
    end
  end


   sum1=zeros(C+1)
   for i=1:C+1   
    for j=1:C+1
     sum1[i]=sum1[i].+T_PM[i,j]
    end
   end
        
        
#Stationary probability matrix
        
  @constraint(opt,[j=1:C.+1],(pi_s[1,j]==(sum(pi_s[1,i].*T_PM[i,j] for i=1:(C.+1)))))
  @constraint(opt,[i=1:1],(sum(pi_s[i,j] for j=1:((C.+1))))==1)
  #@constraint(opt,[i=1:1,j=1:1],pi[i,j]<=1.0)
  #@constraint(opt,[i=1:1],(sum(pi[i,j] for j=((24*(C+1))-(C-1)):24*(C+1)))>=0.999)
        
 optimize!(opt)
    return JuMP.value.(pi_s),JuMP.value.(pi_s[1,1]),JuMP.value.(pi_s[1,C.+1])         
end
(pi_st,pi_z[C],pi_m[C]) = solve_opt(B_C);    
end 

opts=zeros(w)
optsize=0;
for C in 1:w
 if pi_z[C]<=0.1 
  if optsize<=C & optsize==0
  optsize=C-1          
  end                 
 end      
end


#optsize=min(optsi)
#println("wind power data: ",W_P, " MW")
#println("\n")

#println("solar power data: ",S_P, " MW")
#println("\n")

#println("load power data: ",L_P, " MW")
#println("\n")

#println("Diff power data: ",D_P, " MW")
#println("\n")

#println("X Values: ",X, " MW")
#println("\n")

#println("Sum Values: ",sum, " Check")
#println("\n")


#println("Sum Values: ",sum1, " Check")
#println("\n")


#println("Transistion Probability Matrix: ",T_PM, " Matrix")
#println("\n")

#println("Length of data: ",len)
#println("\n")

#println("Stationary Probability: ",pi_st)
#println("\n")

#println("Stationary Probability at state zero for all batteries: ",pi_z)
#println("\n")

println("Stationary Probability at state zero for all batteries: ",pi_z)
println("\n")

println("Stationary Probability at Max state for all batteries: ",pi_m)
println("\n")

println("optimal battery sizes: ",optsize)
println("\n")


#CSV.read("V:\\PHD\\Stochastic_Process\\Project_details\\Literature\\Battery_sizing_data_1.csv");

  Resolving package versions...
No Changes to `C:\Users\pavan\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\pavan\.julia\environments\v1.5\Manifest.toml`


Stationary Probability at state zero for all batteries: [0.24785834983524282, 0.2371287534579457, 0.22801819380617877, 0.2203198653584756, 0.21272519936922218, 0.2062753014408308, 0.1998107008569834, 0.19373269755496575, 0.18809650333549632, 0.18207580837985307, 0.17632331568188916, 0.17103092232773684, 0.16567479749639466, 0.16099016618479733, 0.15574710823246102, 0.15037729267882863, 0.14591845970960637, 0.14162917668685038, 0.13635868090362094, 0.13213056256864475, 0.12755375633114285, 0.12410640612029908, 0.11974732821107575, 0.1166849610448925, 0.11157320091530927, 0.10863505234539023, 0.10512330415291123, 0.10187510015996298, 0.09875441286719187, 0.09530169526792705, 0.09259439744338432, 0.0891857238786996, 0.08666400273241276, 0.08324956455471208, 0.0809961359766949, 0.07838452609394855, 0.07617428830443879, 0.07367724242652367, 0.07136730278907147, 0.0692031549912613, 0.06696881226950768, 0.06498739761587029, 0.06303640534781463, 0.06130596991202942, 0.05989523102983271, 0.0582